In [6]:
# fix, then adopt https://dash-gallery.plotly.host/dash-financial-report/
# sauth --dir dzi_files/ user pass localhost 5555
# https://github.com/thedirtyfew/terracotta-dash-example/blob/master/app.py
# https://github.com/plotly/dash-sample-apps/blob/master/apps/dash-financial-report/app.py
# one terminal launches server, other terminal runs this app, third terminal forwards app; 3 ports
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import dash_table, os
from dash.dependencies import Output, Input


app = JupyterDash(__name__,external_stylesheets=[dbc.themes.COSMO],assets_folder=os.path.abspath("dzi_files"))#'active_learning/round_1/unlabelled/',requests_pathname_prefix='/proxy/{}/'.format(port))

app.config.suppress_callback_exceptions = False

server = app.server

app.layout = html.Div([
    html.Iframe(id='openseadragon',
                src="http://127.0.0.1:5555",width='800',height='800')#os.path.abspath("dzi_files/index.html")
    ])

port=5556
# app._terminate_server_for_port("localhost", port)
app.run_server(debug=True,mode="external",host='localhost',port=port)

Dash app running on http://localhost:5556/


In [7]:
! ls dzi_files

163_A1_1_1_ink.dzi	163_A1_3_1_ink_files	163_A1_5_1_tumor.dzi
163_A1_1_1_ink_files	163_A1_3_1_tumor.dzi	163_A1_5_1_tumor_files
163_A1_1_1_tumor.dzi	163_A1_3_1_tumor_files	163_A1_6_1_ink.dzi
163_A1_1_1_tumor_files	163_A1_4_1_ink.dzi	163_A1_6_1_ink_files
163_A1_2_1_ink.dzi	163_A1_4_1_ink_files	163_A1_6_1_tumor.dzi
163_A1_2_1_ink_files	163_A1_4_1_tumor.dzi	163_A1_6_1_tumor_files
163_A1_2_1_tumor.dzi	163_A1_4_1_tumor_files	index.html
163_A1_2_1_tumor_files	163_A1_5_1_ink.dzi	openseadragon
163_A1_3_1_ink.dzi	163_A1_5_1_ink_files
